In [1]:
from helper import SocketListener, DataQueue
from concurrent.futures import ThreadPoolExecutor
from pynput.keyboard import Key, Controller
import time
import random

In [2]:
keyboard = Controller()
listener = SocketListener()

In [ ]:
# [Miss, Meh, Ok, Good, Great, Perfect, Passed?, Failed?]
hit_counts = [0] * 8
count = 0
keyboard = Controller()
hit = DataQueue()
listener = SocketListener()
  
def process_data():
  while listener.is_listening or listener.is_first_connection:
    if listener.has_connection:
      time_start = time.time()
      hit.clear()
      perform_action()
      data = hit.get()
      if len(data) != 0:
        print("hit type: ", data)

        if data == 6:
          print('Song passed')
          break
        if data == 7:
          print('Song failed')
          break

        print(hit_counts)
        for i in data:
          hit_counts[i] += 1
      else:
        print("no data received")
      
      time_end = time.time()
      time_total = time_end - time_start
      #print(f"it takes {time_total} seconds to fetch data after performing action")

    if not listener.is_first_connection and not listener.has_connection:
        listener.stop()
        break

def random_input():
  inputs = ['s', 'd', 'k', 'l']
  if listener.has_connection:
    inp = random.choice(inputs)
    #print("key input: ",inp)
    keyboard.press(inp)
    time.sleep(0.05)
    keyboard.release(inp)

def perform_action():
  threads = []
  with ThreadPoolExecutor(max_workers=8) as executor:
    for _ in range(4):
      thread = executor.submit(random_input)
      threads.append(thread)

  for thread in threads:
    thread.result()

with ThreadPoolExecutor(max_workers=8) as executor:
    listener.start(data_handler=hit)
    process_data()

Listening on 127.0.0.1:5555
Socket listener stopped.
